In [1]:
import random
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus

In [2]:
df = pd.DataFrame(columns=['activitynum', 'fbfriend', 'igfan', 'joinsa', 'joinballteam',
                           'humorous', 'gpa', 'bodyvalue', 'facevalue', 'popular'])

In [3]:
for i in range(10000):
    tmp = pd.Series({'activitynum':random.randint(0,20), 'fbfriend':random.randint(0,1200), 
                     'igfan':random.randint(0,1200), 'joinsa':random.randint(0,1),
                     'joinballteam':random.randint(0,1),'humorous':random.randint(0,1), 
                     'gpa':random.uniform(0, 4.3), 'bodyvalue':random.randint(1,10), 
                     'facevalue':random.randint(1,10), 'popular':random.randint(0,1)})
    df = df.append(tmp, ignore_index=True)


In [4]:
for i in range(df.shape[0]):
    if df.loc[i,'activitynum'] > 15:
        if df.loc[i,'humorous'] == 1:
            df.at[i,'popular'] = 1
        else:
            if df.loc[i,'facevalue'] > 7:
                df.at[i,'popular'] = 1
            else:
                df.at[i,'popular'] = 0
    elif df.loc[i,'activitynum'] > 8:
        if df.loc[i,'bodyvalue'] > 5:
            if df.loc[i,'joinballteam'] == 1:
                if df.loc[i,'facevalue'] > 7:
                    df.at[i,'popular'] = 1
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0
        else:
            if df.loc[i,'humorous'] == 1:
                if df.loc[i,'facevalue'] > 7:
                    df.at[i,'popular'] = 1
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0
    else:
        if df.loc[i,'bodyvalue'] > 5:
            if df.loc[i,'joinballteam'] == 1:
                if df.loc[i,'facevalue'] > 7:
                    df.at[i,'popular'] = 1
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0
        else:
            if df.loc[i,'facevalue'] > 7:
                df.at[i,'popular'] = 1
            elif df.loc[i,'facevalue'] > 3:
                if df.loc[i,'gpa'] > 4:
                    df.at[i,'popular'] = 1
                elif df.loc[i,'gpa'] > 3:
                    if df.loc[i,'joinsa'] == 1:
                        df.at[i,'popular'] = 1
                    else:
                        df.at[i,'popular'] = 0
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0

In [5]:
df_train = df[:8000]
df_train.head()
df_test = df[8000:]
df_test.head()

y = df_train['popular'].values
df_train = df_train.drop('popular', 1)

In [6]:
dtree=DecisionTreeClassifier(max_depth=4)
dtree.fit(df_train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['Loser','Winner'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree2.pdf")

True

In [7]:
dtree.feature_importances_

array([0.21722706, 0.        , 0.        , 0.        , 0.05160783,
       0.24461579, 0.00596733, 0.10713021, 0.37345178])

In [8]:
y_test = df_test['popular'].values
X_test = df_test.drop('popular', 1)

y_predict = dtree.predict(X_test)

y_predict

array([1., 0., 0., ..., 0., 1., 0.])

In [9]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9625